In [75]:
import googlemaps
import os
from dotenv import load_dotenv
import requests 

load_dotenv()
api_key = os.getenv("GOOGLE_MAPS_API_KEY")
gmaps = googlemaps.Client(key=api_key)

### Places

In [76]:
fields_requested = "places.displayName,"\
                   "places.formattedAddress,"\
                   "places.priceLevel,"\
                   "places.rating,"\
                   "places.userRatingCount,"\
                   "places.regularOpeningHours"
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": api_key,
    "X-Goog-FieldMask": fields_requested
}

json_body = {
    "textQuery": "boba spots near UC Irvine",
}

response = requests.post(
    "https://places.googleapis.com/v1/places:searchText",
    json=json_body,
    headers=headers
)

place_result = response.json()
place_result.keys()

dict_keys(['places'])

In [77]:
# place result is a dictionary with these keys (I think for our project only results useful)
place_result["places"][0].keys()

dict_keys(['formattedAddress', 'rating', 'regularOpeningHours', 'priceLevel', 'userRatingCount', 'displayName'])

In [83]:
one_place = place_result["places"][0]
one_place.get("regularOpeningHours", {}).get("openNow", None)

True

In [85]:
print("Top 3 spots by rating: ")
places = place_result["places"]
places_sorted = sorted(places, key=lambda x: x.get("rating", 0), reverse=True)
for place in places_sorted[:3]:
    name = place.get("displayName", "N/A").get("text", "N/A")
    rating = place.get("rating", "N/A")
    user_rating_count = place.get("userRatingCount", "N/A")
    price_level = place.get("priceLevel", "N/A")

    print(f"{name}: Rating {rating} based on {user_rating_count} reviews (Price Level: {price_level})")

Top 3 spots by rating: 
Ever After Tea Room & Eatery: Rating 4.6 based on 243 reviews (Price Level: PRICE_LEVEL_MODERATE)
Jam Jam Tea Lab - Irvine: Rating 4.6 based on 131 reviews (Price Level: PRICE_LEVEL_INEXPENSIVE)
TP TEA – Irvine Main: Rating 4.5 based on 218 reviews (Price Level: PRICE_LEVEL_INEXPENSIVE)


In [101]:
import json

def format_price(price_enum):
    price_map = {
        "PRICE_LEVEL_FREE": 0,
        "PRICE_LEVEL_INEXPENSIVE": 1,
        "PRICE_LEVEL_MODERATE": 2,
        "PRICE_LEVEL_EXPENSIVE": 3,
        "PRICE_LEVEL_VERY_EXPENSIVE": 4,
    }
    return price_map.get(price_enum, 0)

results_to_save = []
for place in place_result["places"]:
    results_to_save.append({
        "name": place.get("displayName", {}).get("text", "N/A"),
        "open_now": place.get("regularOpeningHours", {}).get("openNow", None),
        "price_level": format_price(place.get("priceLevel", "unspecified")),
        "rating": place.get("rating"),
        "user_ratings_total": place.get("userRatingCount")
    })

with open("places_data.json", "w") as f:
    json.dump(results_to_save, f, indent=2)

In [ ]:
PRICE_LEVEL_MAP = {
    "PRICE_LEVEL_FREE": 0,
    "PRICE_LEVEL_INEXPENSIVE": 1,
    "PRICE_LEVEL_MODERATE": 2,
    "PRICE_LEVEL_EXPENSIVE": 3,
    "PRICE_LEVEL_VERY_EXPENSIVE": 4,
}

m = 50    # minimum ratings threshold

def bayesian_avg(rating, num_ratings, C):
    sum_R = rating * num_ratings
    return (C * m + sum_R) / (m + num_ratings)

def compute_weighted_score(bayesian_score, price_level, budget):
    price_level = price_level or 1
    budget_match = 1 if price_level <= budget else 0
    return 0.9 * bayesian_score + 0.1 * budget_match

def extract_place_info(place, budget, C):
    rating = place.get("rating", 0)
    num_ratings = place.get("userRatingCount", 0)
    price_level = PRICE_LEVEL_MAP.get(place.get("priceLevel"))
    b_score = bayesian_avg(rating, num_ratings, C)
    return {
        "name": place.get("displayName", {}).get("text"),
        "open_now": place.get("regularOpeningHours", {}).get("openNow"),
        "price_level": price_level,
        "rating": rating,
        "user_ratings_total": num_ratings,
        "bayesian_score": b_score,
        "weighted_score": compute_weighted_score(b_score, price_level, budget),
    }

def search_places(query, budget):
    """Call v2 searchText, extract and score places, return sorted by weighted_score."""
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": (
            "places.displayName,"
            "places.formattedAddress,"
            "places.priceLevel,"
            "places.rating,"
            "places.userRatingCount,"
            "places.regularOpeningHours"
        ),
    }
    response = requests.post(
        "https://places.googleapis.com/v1/places:searchText",
        json={"textQuery": query},
        headers=headers,
    )
    places = response.json().get("places", [])

    # C = mean rating of this result set
    ratings = [p.get("rating", 0) for p in places]
    C = sum(ratings) / len(ratings) if ratings else 0

    results = [extract_place_info(p, budget, C) for p in places]
    results.sort(key=lambda x: x["weighted_score"], reverse=True)
    return results

In [ ]:
# quick test: boba spots near UCI, budget 2
results = search_places("boba spots near UC Irvine", budget=2)
for r in results[:5]:
    price_str = "$" * r["price_level"] if r["price_level"] else "N/A"
    print(f"{r['name']}: weighted={r['weighted_score']:.2f}  rating={r['rating']}  price={price_str}")

### Distance

In [94]:
origin = place_result.get("places")[0].get("formattedAddress")
destination = place_result.get("places")[7].get("formattedAddress")
print("Start: ", origin)
print("End: ", destination)

Start:  4199 Campus Dr suite c, Irvine, CA 92612, USA
End:  92 Corporate Park suite d, Irvine, CA 92606, USA


In [108]:
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": api_key,
    "X-Goog-FieldMask": "routes.distanceMeters,routes.duration"
}

json_body = {
    "origin": {
        "address": origin
    },
    "destination": {
        "address": destination
    },
    "travelMode": "DRIVE"
}

response = requests.post(
    "https://routes.googleapis.com/directions/v2:computeRoutes",
    json=json_body,
    headers=headers
)

data = response.json()
print(data)

{'routes': [{'distanceMeters': 5587, 'duration': '626s'}]}
